In [3]:
pip install ortools requests folium

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
import folium
import json
from math import radians, sin, cos, sqrt, atan2
import requests

In [5]:
# Function to calculate distance between two points using Haversine formula
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Calculate the differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Calculate distance using Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [6]:
# Function to calculate distance matrix
def calculate_distance_matrix(locations):
    num_locations = len(locations)
    distance_matrix = []

    for i in range(num_locations):
        row = []
        for j in range(num_locations):
            if i == j:
                row.append(0)  # Distance from a location to itself is 0
            else:
                lat1, lon1 = locations[i]
                lat2, lon2 = locations[j]
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                row.append(int(round(distance, 2)*1000))  # Round distance to 2 decimal places
        distance_matrix.append(row)

    return distance_matrix

In [7]:
# Latitude and longitude values for 7 locations in Chennai
chennai_locations = [
    (13.08726, 80.2739),  # Anna Nagar
    (13.08498, 80.2707),  # Kilpauk
    (13.09182, 80.2707),  # Chetpet
    (13.0827, 80.2715),   # Egmore
    (13.08384, 80.2715),  # Nungambakkam
    (13.0884, 80.2723),   # Thousand Lights
    (13.09068, 80.2723)   # Teynampet
]

# Calculate the distance matrix
distance_matrix = calculate_distance_matrix(chennai_locations)

# Print the distance matrix
for row in distance_matrix:
    print(row)

[0, 430, 610, 570, 460, 210, 420]
[430, 0, 760, 270, 150, 420, 660]
[610, 760, 0, 1020, 890, 420, 210]
[570, 270, 1020, 0, 130, 640, 890]
[460, 150, 890, 130, 0, 510, 770]
[210, 420, 420, 640, 510, 0, 250]
[420, 660, 210, 890, 770, 250, 0]


In [8]:
def create_data_model():
    #data from the co-ordinates
    data = {}
    data["distance_matrix"] = calculate_distance_matrix(chennai_locations)
    data["num_vehicles"] = 3
    data["depot"] = 0
    return data

In [9]:
def print_solution(data, manager, routing, solution):
    #printing the routes
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    routes_sol=[]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        route.append(manager.IndexToNode(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        print(f"Route for vehicle {vehicle_id}: {route}, Distance of the route: {route_distance}m")
        routes_sol.append(route)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")
    return routes_sol


In [10]:
def get_route_details(data, manager, routing, solution):
    routes = []
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

In [11]:
def get_route(start, end, tomtom_api_key):
    url = f'https://api.tomtom.com/routing/1/calculateRoute/{start[0]},{start[1]}:{end[0]},{end[1]}/json?key={tomtom_api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        route_data = response.json()
        route_geometry = route_data.get('routes')[0].get('legs')[0].get('points')
        if route_geometry:
            route_points = [(point['latitude'], point['longitude']) for point in route_geometry]
            return route_points
        else:
            print("No route points found in the response:", route_data)
            return None
    else:
        print("Failed to fetch route:", response.text)
        return None

In [12]:
def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # Get the route details
        routes = get_route_details(data, manager, routing, solution)

        # Print solution details
        routes_sol=print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")
    # TomTom API key
    tomtom_api_key = 'qN86js1EGFaSWvQ28TASgkUuphaxAnbF'

    # Initialize the map
    mymap = folium.Map(location=[13.08726, 80.2739], zoom_start=14)

    # Plot the coordinates on the map
    for i, location in enumerate(chennai_locations):
        folium.Marker(location, popup=f'Location {i}').add_to(mymap)

    # Define colors for each route
    colors = ['red', 'green', 'blue']

    # Get and plot the navigation between each route
    for route, color in zip(routes_sol, colors):
        for i in range(len(route) - 1):
            start_index = route[i]
            end_index = route[i + 1]
            start = chennai_locations[start_index]
            end = chennai_locations[end_index]
            route_points = get_route(start, end, tomtom_api_key)
            if route_points:
                folium.PolyLine(route_points, color=color).add_to(mymap)

    # Save the map to an HTML file
    mymap.save('navigation_between_routes.html')


In [13]:
if __name__ == "__main__":
    main()

Objective: 127460
Route for vehicle 0: [0, 1, 5, 0], Distance of the route: 1270m
Route for vehicle 1: [0, 6, 2, 0], Distance of the route: 1850m
Route for vehicle 2: [0, 4, 3, 0], Distance of the route: 1730m
Maximum of the route distances: 1850m


SAMPLE ROUGH GRID

In [14]:
import folium

# Set up Folium map with TomTom as the tile provider
tomtom_api_key = 'qN86js1EGFaSWvQ28TASgkUuphaxAnbF'

# Get the route between two coordinates from TomTom API
def get_route(start, end, tomtom_api_key):
    url = f'https://api.tomtom.com/routing/1/calculateRoute/{start[0]},{start[1]}:{end[0]},{end[1]}/json?key={tomtom_api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        route_data = response.json()
        route_geometry = route_data.get('routes')[0].get('legs')[0].get('points')
        if route_geometry:
            route_points = [(point['latitude'], point['longitude']) for point in route_geometry]
            return route_points
        else:
            print("No route points found in the response:", route_data)
            return None
    else:
        print("Failed to fetch route:", response.text)
        return None

my_map = folium.Map(location=[13.08726, 80.2739], zoom_start=14, tiles=f"https://api.tomtom.com/map/1/tile/basic/main/{{z}}/{{x}}/{{y}}.png?key={tomtom_api_key}", attr='Map data © TomTom')

# Add markers to the map for each location
locations = [
    (13.08726, 80.2739),  # Location 0
    (13.08498, 80.2707),  # Location 1
    (13.09182, 80.2707),  # Location 2
    (13.0827, 80.2715),   # Location 3
    (13.08384, 80.2715),  # Location 4
    (13.0884, 80.2723),   # Location 5
    (13.09068, 80.2723)   # Location 6
]

for i in range(len(locations)):
    folium.Marker(location=locations[i], popup=f'Location {i}').add_to(my_map)

# Define the routes
routes = [
    [0, 1, 5, 0],  # Route for vehicle 0
    [0, 6, 2, 0],  # Route for vehicle 1
    [0, 4, 3, 0]   # Route for vehicle 2
]

# Add polylines representing each route
colors = ['blue', 'red', 'green']  # Define colors for each route
for i, route in enumerate(routes):
    route_locations = [locations[idx] for idx in route]
    folium.PolyLine(route_locations, color=colors[i], weight=2.5, opacity=1).add_to(my_map)

# Save the map as an HTML file
my_map.save("map_rough.html")